<img src="https://www.rp.edu.sg/images/default-source/default-album/rp-logo.png" width="200" alt="Republic Polytechnic"/>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/koayst-rplesson/SST_DP2025/blob/main/Day_03/L16/L16-01-dataset_upload-answer.ipynb)

# Dataset Upload

In addition to creating and editing Datasets in the LangSmith UI, you can also create and edit datasets with the LangSmith SDK.

Let's go ahead an upload a list of examples that we have from our RAG application to LangSmith as a new dataset.

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain
%pip install --quiet -U langgraph
%pip install --quiet -U langchain-openai
# %pip install --quiet -U grandalf
%pip install --quiet -U langchain-community
# %pip install --quiet -U faiss-cpu
# %pip install --quiet -U pytube
# %pip install --quiet -U youtube-transcript-api
%pip install --quiet -U python-dotenv

In [ ]:
import sys
# map google drive
from google.colab import drive
drive.mount('/content/drive')
gdrive_path = "/content/drive/My Drive/Colab Notebooks/00-LLM App Dev/Day 3"
import os
os.chdir(gdrive_path)
sys.path.append(gdrive_path)
!pwd

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/00-LLM App Dev/Day 3


In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-rp"

In [ ]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

In [ ]:
print(f'OpenAI API Key: {os.environ["OPENAI_API_KEY"]}')
print(f'LangSmith API Key: {os.environ["LANGCHAIN_API_KEY"]}')
print(f'Langsmith project: {os.environ["LANGCHAIN_PROJECT"]}')

NameError: name 'os' is not defined

In [ ]:
from langsmith import Client

example_inputs = [
("How do I set up tracing to LangSmith if I'm using LangChain?", "To set up tracing to LangSmith while using LangChain, you need to set the environment variable `LANGCHAIN_TRACING_V2` to 'true'. Additionally, you must set the `LANGCHAIN_API_KEY` environment variable to your API key. By default, traces will be logged to a project named \"default.\""),
("How can I trace with the @traceable decorator?", "To trace with the @traceable decorator in Python, simply decorate any function you want to log traces for by adding `@traceable` above the function definition. Ensure that the LANGCHAIN_TRACING_V2 environment variable is set to 'true' to enable tracing, and also set the LANGCHAIN_API_KEY environment variable with your API key. By default, traces will be logged to a project named \"default,\" but you can configure it to log to a different project if needed."),
("How do I pass metadata in with @traceable?", "You can pass metadata with the @traceable decorator by specifying arbitrary key-value pairs as arguments. This allows you to associate additional information, such as the execution environment or user details, with your traces. For more detailed instructions, refer to the LangSmith documentation on adding metadata and tags."),
("What is LangSmith used for in three sentences?", "LangSmith is a platform designed for the development, monitoring, and testing of LLM applications. It enables users to collect and analyze unstructured data, debug issues, and create datasets for testing and evaluation. The tool supports various workflows throughout the application development lifecycle, enhancing the overall performance and reliability of LLM applications."),
("What testing capabilities does LangSmith have?", "LangSmith offers capabilities for creating datasets of inputs and reference outputs to run tests on LLM applications, supporting a test-driven approach. It allows for bulk uploads of test cases, on-the-fly creation, and exporting from application traces. Additionally, LangSmith facilitates custom evaluations to score test results, enhancing the testing process."),
("Does LangSmith support online evaluation?", "Yes, LangSmith supports online evaluation as a feature. It allows you to configure a sample of runs from production to be evaluated, providing feedback on those runs. You can use either custom code or an LLM as a judge for the evaluations."),
("Does LangSmith support offline evaluation?", "Yes, LangSmith supports offline evaluation through its evaluation how-to guides and features for managing datasets. Users can manage datasets for offline evaluations and run various types of evaluations, including unit testing and auto-evaluation. This allows for comprehensive testing and improvement of LLM applications."),
("Can LangSmith be used for finetuning and model training?", "Yes, LangSmith can be used for fine-tuning and model training. It allows you to capture run traces from your deployment, query and filter this data, and convert it into a format suitable for fine-tuning models. Additionally, you can create training datasets to keep track of the data used for model training."),
("Can LangSmith be used to evaluate agents?", "Yes, LangSmith can be used to evaluate agents. It provides various evaluation strategies, including assessing the agent's final response, evaluating individual steps, and analyzing the trajectory of tool calls. These methods help ensure the effectiveness of LLM applications."),
("How do I create user feedback with the LangSmith sdk?", "To create user feedback with the LangSmith SDK, you first need to run your application and obtain the `run_id`. Then, you can use the `create_feedback` method, providing the `run_id`, a feedback key, a score, and an optional comment. For example, in Python, it would look like this: `client.create_feedback(run_id, key=\"feedback-key\", score=1.0, comment=\"comment\")`."),
]

client = Client()
# TODO: Fill in dataset id
dataset_id = ""

# Prepare inputs and outputs for bulk creation
inputs = [{"question": input_prompt} for input_prompt, _ in example_inputs]
outputs = [{"output": output_answer} for _, output_answer in example_inputs]

client.create_examples(
  inputs=inputs,
  outputs=outputs,
  dataset_id=dataset_id,
)

Let's have a look in langsmith and tag this version

## Submitting another Trace

I've moved our RAG application definition to `app.py` so we can quickly import it.

In [ ]:
from app import langsmith_rag

Fetching pages: 100%|##########| 219/219 [00:07<00:00, 28.34it/s]


Let's ask another question to create a new trace!

In [ ]:
question = "How do I set up tracing to LangSmith if I'm using LangChain?"
langsmith_rag(question)

'To set up tracing to LangSmith with LangChain, first, install LangSmith using `pip install -U langsmith openai` or `yarn add langsmith openai`. Next, create an API key on the Settings page, and set up your environment by exporting `LANGCHAIN_TRACING_V2=true`, `LANGCHAIN_API_KEY=<your-api-key>`, and `OPENAI_API_KEY=<your-openai-api-key>`. Finally, ensure the tracing is enabled by confirming the required environment variables are set correctly.'

Navigate to Runs, you can add the input and output to your dataset.

It is also very important to be able to create unit tests for each of the individual steps within the application.

A good practice is to create seperate dataset for both the internal sub runs here. Create a dataset specifically to evaluate our retrieve_documents step to see if our retrived documents are relavent and useful.  And then create a dataset specifically for generate_response so that we know given good document and reasonable questions, how good are we at generating a good response.